In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import scipy.optimize as optimize
import datetime

plt.rcParams['font.size'] = 14

# Project:  writing and testing an algorithm to pick initial parameters for a fit

Recall that in notebook '06_02_SDSS_Specta' we did fit to a spectral line in the SDSS data that included both a  signal and a background part in the model.

Recall also that we picked initial values for the fit parameters looking at the plot at estimating values, and that there was a question about how you might write an algorithm to guess the initial parameters automatically.

This is your chance to do that.

This notebook has the pieces you will need to do that.

Some goals for this project might include:

1. Write a function to extract initial guesses for the fit parameters.

2. Test the function out on a few different peaks

3. Estiamte how close you have to be to the correct values of the parameters for the fitter to succeed. 

4. (optional) Have your code automatically identify peaks within the spectrum and fit each individually in sequence using the techniques you developed 

### Cell to read in the data and put it into two arrays

In [2]:
data = np.loadtxt(open("../data/sdss_galaxy.txt", 'rb'), usecols=range(4))

## This is how we pull out the data from columns in the array.

## They put the data in Angstroms, lets use nano-meters instead, 1 Angstrom = 0.1 nm. 
wavelength = data[:,0] / 10.
flux = data[:,1]
#best_fit = data[:,2]
#sky_flux = data[:,3]

FileNotFoundError: [Errno 2] No such file or directory: '../data/sdss_galaxy.txt'

### Function to select a sub-set of the data over a smaller set of wavelengths

In [ ]:
def selectData(wavelength, flux, lamb_lo, lamb_hi):
    idx_lo = np.argmin(np.abs(wavelength - lamb_lo))
    idx_hi = np.argmin(np.abs(wavelength - lamb_hi))
    return wavelength[idx_lo:idx_hi], flux[idx_lo:idx_hi]

### Functions used to define the model and the cost function

In [ ]:
from functools import partial

def Gauss(x, prefact, mu, sigma):
    return prefact*stats.norm(loc=mu, scale=sigma).pdf(x)

def poly1(x, ref_lambda, offset, slope):
    return offset + (x-ref_lambda)*slope

def model_func(x, ref_lambda, prefact, mu, sigma, offset, slope):
    return Gauss(x, prefact, mu, sigma) + poly1(x, ref_lambda, offset, slope)

def generic_chi2(params, data_vals, model, x, ref_lambda):
    model_vals = model(x, ref_lambda, *params)
    return np.sum((data_vals - model_vals)**2)

def cost_func(data_vals, model, x, ref_lambda):
    return partial(generic_chi2, data_vals=data_vals, model=model, x=x, ref_lambda=ref_lambda)

### Function to overplot the model on the data

In [ ]:
def plotModels(cutout_wl, cutout_flux, ref_lambda, init_pars):
    
    model_vals = model_func(cutout_wl, ref_lambda, *init_pars)
    background_vals = poly1(cutout_wl, ref_lambda, init_pars[3], init_pars[4])

    _ = plt.plot(cutout_wl, cutout_flux, label="data")
    _ = plt.plot(cutout_wl, background_vals, label="background model")
    _ = plt.plot(cutout_wl, model_vals, label="model")
    _ = plt.xlabel(r'$\lambda [nm]$')
    _ = plt.ylabel(r'Flux [arcane units]')
    _ = plt.legend()

### Function to define the cost function, fit the data, and plot the result

In [ ]:
def fitAndPlotResult(cutout_wl, cutout_flux, ref_lambda, init_pars):
    our_cost_func = cost_func(cutout_flux, model_func, cutout_wl, ref_lambda=ref_lambda)
    result = optimize.minimize(our_cost_func, x0=np.array(init_pars))
    fit_pars = result['x']
    model_fit = model_func(cutout_wl, ref_lambda, *fit_pars)
    background_fit = poly1(cutout_wl, ref_lambda, fit_pars[3], fit_pars[4])
    print("Best Fit ---------")
    print("Line Intensity: %.1f [arcane units]" % fit_pars[0])
    print("Line Peak: %.4f [nm]" % fit_pars[1])
    print("Line Width: %.4f [nm]" % fit_pars[2])
    print("Background at 500 nm: %.2f [arcane units]" % fit_pars[3])
    print("Background slope: %.2f [arcane_units / nm]" % fit_pars[4])
    _ = plt.plot(cutout_wl, cutout_flux, label="data")
    _ = plt.plot(cutout_wl, background_fit, label="background model")
    _ = plt.plot(cutout_wl, model_fit, label="full model")
    _ = plt.xlabel(r"$\lambda [nm]$")
    _ = plt.ylabel("Flux [arcane units]")
    _ = plt.legend()

# Lets have a look at the complete spectrum

In [ ]:
_ = plt.plot(wavelength, flux)
_ = plt.xlabel(r"$\lambda [nm]$")
_ = plt.ylabel("Flux [arcane units]")
_ = plt.title("A spectrum from SDSS")

# Now let's select a cutout of the data and have a look at that

In [ ]:
cutout_wl, cutout_flux = selectData(wavelength, flux, 480, 515)
_ = plt.plot(cutout_wl, cutout_flux)
_ = plt.xlabel(r"$\lambda [nm]$")
_ = plt.ylabel("Flux [arcane units]")

# Setting the reference wavelenght (i.e., the reference point for the background line)

Because we want to be able to look at other peaks as well, and because we learned that if we want to fit a line it is smart to pick a reference x-axis value in the middle of the data, I've changed the model function a tiny bit, to 
let you set the reference point, if you look at the cell that defines the model you can see that the polynomical is 
`offset + (x-ref_lambda)*slope` instead of `offset + (x-500)*slope` as it was in the original notebook.



In [ ]:
ref_lambda = 500

# Setting the intial guess parameters

In [ ]:
prefact_0 = 235.
mu_0 = 505.8
sigma_0 = 0.25
offset_0 = 60
slope_0 = 0

init_pars = (prefact_0, mu_0, sigma_0, offset_0, slope_0)

# Plotting the initial guess model and the data

In [ ]:
plotModels(cutout_wl, cutout_flux, ref_lambda, init_pars)

# Fitting the data an replotting the result

In [ ]:
fitAndPlotResult(cutout_wl, cutout_flux, ref_lambda, init_pars)

# A function that might be useful: `np.argmax`

In [ ]:
print("The index of the largest bin is: ", np.argmax(cutout_flux))
print("The flux value in the largest bin is: ", cutout_flux[np.argmax(cutout_flux)])